In [285]:
from multiview.embed.gcca import GCCA
import matplotlib.pyplot as plt
import numpy as np
import scipy
%matplotlib inline
import seaborn as sns
from scipy.sparse.linalg import svds

In [390]:
## Views with same features of high/low variance

np.random.seed(0)

def get_train_test(n=100, mu=0, var=1, nviews=3,m=1000):
    np.random.seed(0)
    
    X_TRAIN = np.random.normal(mu,var,(n,1))
    X_TEST = np.random.normal(mu,var,(n,1))

    Xs_train = []
    Xs_test = []
    for i in range(nviews):
        X_train = np.hstack((np.random.normal(0,1,(n,i)), 
                             X_TRAIN, 
                             np.random.normal(0,1,(n,m-2-i)),
                             np.random.normal(0,var,(n,1))
                            ))
        X_test = np.hstack((np.random.normal(0,1,(n,i)), 
                            X_TEST, 
                            np.random.normal(0,1,(n,m-2-i)),
                            np.random.normal(0,var,(n,1))
                           ))
        
        Xs_train.append(X_train)
        Xs_test.append(X_test)
    
    return(Xs_train,Xs_test)

## Positive Test
1 high variance shared feature, 1 hight variance noise

In [391]:
nviews = 3
Xs_train, Xs_test = get_train_test(var=10,nviews=nviews,m=1000)

In [392]:
gcca = GCCA()
gcca.fit(Xs_train, n_components=2)
Xs_hat = gcca.transform(Xs_test)

In [393]:
np.corrcoef(np.array(Xs_hat)[:,:,0])

array([[1.        , 0.99698235, 0.99687182],
       [0.99698235, 1.        , 0.99689792],
       [0.99687182, 0.99689792, 1.        ]])

In [394]:
Xs_hat = []
for i in range(len(Xs_train)):
    _,_,vt = svds(Xs_train[i],k=1)
    Xs_hat.append(Xs_test[i] @ vt.T)

In [395]:
np.corrcoef(np.array(Xs_hat)[:,:,0])

array([[ 1.        , -0.54014795,  0.51173297],
       [-0.54014795,  1.        , -0.98138902],
       [ 0.51173297, -0.98138902,  1.        ]])

## Negative Test
1 low variance shared feature

In [406]:
nviews = 3
Xs_train, Xs_test = get_train_test(var=1,nviews=nviews,m=1000)

In [407]:
gcca = GCCA()
gcca.fit(Xs_train, n_components = 2)
Xs_hat = gcca.transform(Xs_test)

In [408]:
np.corrcoef(np.array(Xs_hat)[:,:,0])

array([[ 1.        ,  0.31254995, -0.02208907],
       [ 0.31254995,  1.        ,  0.13722633],
       [-0.02208907,  0.13722633,  1.        ]])

In [409]:
Xs_hat = []
for i in range(len(Xs_train)):
    _,_,vt = svds(Xs_train[i],k=1)
    Xs_hat.append(Xs_test[i] @ vt.T)

In [410]:
np.corrcoef(np.array(Xs_hat)[:,:,0])

array([[1.        , 0.01016507, 0.0888701 ],
       [0.01016507, 1.        , 0.03812276],
       [0.0888701 , 0.03812276, 1.        ]])